In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 day 0 hours 13 mins
H2O_cluster_timezone:,Asia/Tokyo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_root_iycqf9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.617 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [3]:
train = h2o.get_frame("people_train")
valid = h2o.get_frame("people_valid")
test = h2o.get_frame("people_test")

In [4]:
#Printing out the # of rows for each data set.
print( "%d/%d/%d" % ( train.nrows, valid.nrows, test.nrows ) )

788/118/94


In [5]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [6]:
y = "income"
ignoreFields = [ y, "id" ]
x = [ i for i in train.names if i not in ignoreFields ]

In [7]:
m1 = H2OGradientBoostingEstimator(model_id = "defaults")
m1.train(x, y, train,
         validation_frame = valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [8]:
#Displaying MAE (mean absolute error) of training data set.
m1.mae(train = True)

1006.066212920368

In [9]:
m1.mae(valid = True)

1309.2480756799962

In [10]:
perf = m1.model_performance(test)
perf.mae()

1197.269856250526

In [11]:
#Overfitting GBM model built above.
m2 = H2OGradientBoostingEstimator(model_id = "overfit" ,
                                                          ntrees = 1000,
                                                          max_depth = 10)
m2.train( x, y, train,
         validation_frame = valid )

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [14]:
print( "Train: %d  - ->  %d" % (m1.mae(train = True), m2.mae(train = True) ))
print( "Valid: %d  - ->  %d" % (m1.mae(valid = True), m2.mae(valid = True) ))
print( " Test: %d  - ->  %d" % (perf.mae(), m2.model_performance(test).mae() ))

Train: 1006  - ->  351
Valid: 1309  - ->  1533
 Test: 1197  - ->  1396
